# Notebook to download pdfs from links

In [255]:
import pandas as pd
import numpy as np
import openpyxl
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List
import PyPDF2
from PyPDF2 import PdfReader
from bs4 import BeautifulSoup
import requests
import xlsxwriter
import pdfminer
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
import io

from datetime import date
TODAY_STR = date.today().strftime("%d%m%y")

# Set max column widths
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

/var/folders/rj/t9tbtlf12d9_bn6nhb1f0tkc0000gq/T/ipykernel_81480/2843047389.py:25: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


In [256]:
# Import data
DATE_STR = "121022"
industryPDFs_df = pd.read_excel(f"/Users/thomas/Documents/BEIS/scraper/scraped_data/{DATE_STR}-industry_PDF.xlsx", engine = "openpyxl").drop(columns = "Unnamed: 0")
topicsPDFs_df = pd.read_excel(f"/Users/thomas/Documents/BEIS/scraper/scraped_data/{DATE_STR}-topics_PDFs.xlsx", engine = "openpyxl").drop(columns = "Unnamed: 0")
COSHH_PDFs_df = pd.read_excel(f"/Users/thomas/Documents/BEIS/scraper/scraped_data/{DATE_STR}-COSHH_PDFs.xlsx", engine = "openpyxl").drop(columns = "Unnamed: 0")
publications_PDFs_df = pd.read_excel(f"/Users/thomas/Documents/BEIS/scraper/scraped_data/{DATE_STR}-Publications_PDFs.xlsx", engine = "openpyxl").drop(columns = "Unnamed: 0")
catelogue_PDFs_df = pd.read_excel(f"/Users/thomas/Documents/BEIS/scraper/scraped_data/{DATE_STR}-catelogue_PDFs.xlsx", engine = "openpyxl").drop(columns = "Unnamed: 0")
construction_PDFs_df = pd.read_excel(f"/Users/thomas/Documents/BEIS/scraper/scraped_data/{TODAY_STR}-construction_PDFs.xlsx", engine = "openpyxl").drop(columns = "Unnamed: 0")

In [257]:
# Column to say which dataframe it's from
def namestr(obj, namespace):
    return [name for name in namespace if namespace[name] is obj]
    
list_of_dfs = [industryPDFs_df, topicsPDFs_df, COSHH_PDFs_df, publications_PDFs_df, catelogue_PDFs_df, construction_PDFs_df]
for df in list_of_dfs:
    df["Source"] = namestr(df, globals())[0]

# Combine dfs and drop duplicates based on pdf name
combined_PDF_df = pd.concat([industryPDFs_df, topicsPDFs_df, COSHH_PDFs_df, publications_PDFs_df, catelogue_PDFs_df, construction_PDFs_df]).drop_duplicates(subset = "PDFs", keep = "last").reset_index(drop = True)

print(f"Number of unique pdfs is {len(combined_PDF_df)}")

Number of unique pdfs is 927


### Many PDF links haven't been constructed properly due to relative links so are constructed now

In [262]:
# Replace where there are double forward slashes in URLs
combined_PDF_df["URLs"] = combined_PDF_df["URLs"].apply(lambda x : x.replace("//", "/"))
combined_PDF_df["URLs"] = combined_PDF_df["URLs"].apply(lambda x : x.replace("https:/www.hse.gov.uk/", "https://www.hse.gov.uk/"))

In [263]:
# Try to call pdf, if you can't then add string after https://www.hse.gov.uk/ from URL to the PDFs
for loc in combined_PDF_df.index:
    if requests.get(combined_PDF_df["PDFs"].iloc[loc]).status_code == 404:
        # If https appears twice, get rid of 'https://www.hse.gov.uk/'
        counter = 0
        for string in combined_PDF_df["PDFs"].iloc[loc].split("/"):
            if "https:" == string or "http:" == string:
                counter += 1
        if counter == 2:
            combined_PDF_df["PDFs"].iloc[loc] = combined_PDF_df["PDFs"].iloc[loc].replace("https://www.hse.gov.uk/", "", 1)
        # If there is no / after pubns, add it
        elif "pubns" in combined_PDF_df["PDFs"].iloc[loc] and "pubns/" not in combined_PDF_df["PDFs"].iloc[loc]:
            combined_PDF_df["PDFs"].iloc[loc] = combined_PDF_df["PDFs"].iloc[loc].replace("pubns", "pubns/")
        else:
            combined_PDF_df["PDFs"].iloc[loc] = 'https://www.hse.gov.uk/' + combined_PDF_df["URLs"].iloc[loc].split("/")[3] + "/" + combined_PDF_df["PDFs"].iloc[loc].replace('https://www.hse.gov.uk/', "")
    # Replace double pubns if they appear
        second_counter = 0
        for string in combined_PDF_df["PDFs"].iloc[loc].split("/"):
            if "pubns" == string:
                second_counter += 1
        if counter == 2:
            combined_PDF_df["PDFs"].iloc[loc] = combined_PDF_df["PDFs"].iloc[loc].replace("pubns/pubns/", "pubns/")
        if combined_PDF_df["PDFs"].iloc[loc][0] != "h":
            combined_PDF_df["PDFs"].iloc[loc]  = "/".join(combined_PDF_df["PDFs"].iloc[loc].split("/")[1:])



In [264]:
# Get a list of all pdfs which can't be accessed still
list_of_failed_links = [pdf for pdf in combined_PDF_df["PDFs"] if requests.get(pdf).status_code == 404]

### For now I am ignoring failed links pdfs in the interests of time

In [265]:
# Cut failed links from the dataframe
pdfs_to_download = combined_PDF_df[~combined_PDF_df["PDFs"].isin(list_of_failed_links)]

In [266]:
# Download pdfs to /Users/thomas/Documents/BEIS/input_data/pdfs
# importing PdfFileWriter class

from pathlib import Path
import os

for pdf in pdfs_to_download["PDFs"]:
    response = requests.get(pdf)
    with open(os.path.join("/Users/thomas/Documents/BEIS/input_data/pdfs/" + pdf.replace("/", "rand_str")), "wb") as file:
        file.write(response.content)

In [271]:
# Read pdfs from the directory and output to a dataframe
def read_pdf_from_directory(dir)-> pd.DataFrame:
    """
    Reads data from pdfs and adds a column in the df with the textual data
    """
    pdf_columns = []
    for pdf in os.listdir(dir):
        file = os.path.join(dir, pdf)
        try:
            i_f = open(file,'rb')
            resMgr = PDFResourceManager()
            retData = io.StringIO()
            txtConverter = TextConverter(resMgr,retData, laparams= LAParams())
            interpreter = PDFPageInterpreter(resMgr, txtConverter)
            for page in PDFPage.get_pages(i_f):
                interpreter.process_page(page)
            txt = retData.getvalue() 
            string_txt = str(txt)
            pdf_columns.append({"PDFs" : pdf, "Secondary Legislation" : string_txt}) 
        except:
            continue
    pdf_df = pd.DataFrame(pdf_columns)
    pdf_df["PDFs"] = pdf_df["PDFs"].apply(lambda x: x.replace("rand_str", "/"))
    return pdf_df

pdfminer_df = read_pdf_from_directory("/Users/thomas/Documents/BEIS/input_data/pdfs/")

In [272]:
# Get additional pdfs
additional_pdfminer_df = read_pdf_from_directory("/Users/thomas/Documents/BEIS/input_data/additional_pdfs/")
additional_pdfminer_df["Source"] = "Health and Safety Law"

In [273]:
# Copy PDF df and add column for urls
final_df = pd.concat([pdfminer_df, additional_pdfminer_df])
final_df = final_df.merge(combined_PDF_df, how = "left", on = "PDFs")

In [274]:
# Output to dataframe
final_df.to_excel(f"/Users/thomas/Documents/BEIS/input_data/{TODAY_STR}-pdf_rawdata.xlsx", engine='xlsxwriter')